# Convexidad del Modelo Logístico

Demostración práctica de la ventaja de la convexidad (Proposición 2.1) usando el dataset de cáncer de mama de *scikit‑learn*. Se compara la estabilidad de la regresión logística (convexa) con una red neuronal MLP (no convexa).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import statsmodels.api as sm


In [ ]:
# 1. Cargar y simplificar dataset (2 features para visualización)
data = load_breast_cancer(as_frame=True)
df = data.frame
X_raw = df[data.feature_names].values[:, :2]   # sólo 2 primeras columnas
y = df["target"].values

# Estandarizar + intercepto
X_scaled = StandardScaler().fit_transform(X_raw)
X = np.column_stack([np.ones_like(y), X_scaled])  # (n,3)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# --------- Funciones logísticas (convexas) -------------
def nll(beta, X, y):
    z = X @ beta
    return np.sum(np.logaddexp(0, z) - y * z)

def grad_nll(beta, X, y):
    z = X @ beta
    p = 1 / (1 + np.exp(-z))
    return X.T @ (p - y)

def gradient_descent(beta0, X, y, lr=0.01, max_iter=10000, tol=1e-6):
    beta = beta0.copy()
    for _ in range(max_iter):
        g = grad_nll(beta, X, y)
        beta_new = beta - lr * g
        if np.linalg.norm(g) < tol:
            break
        beta = beta_new
    return beta

betas = []
for seed in range(5):
    rng = np.random.default_rng(seed)
    beta0 = rng.standard_normal(X.shape[1])
    betas.append(gradient_descent(beta0, X_train, y_train))

betas = np.vstack(betas)
beta_ref = betas[0]
coef_diff = np.linalg.norm(betas - beta_ref, axis=1)

def predict(beta, X):
    return (X @ beta > 0).astype(int)

acc_log = [accuracy_score(y_test, predict(b, X_test)) for b in betas]

logistic_df = pd.DataFrame({
    "seed": range(5),
    "coef_intercept": betas[:, 0],
    "coef_x1": betas[:, 1],
    "coef_x2": betas[:, 2],
    "||β − β₀||₂": coef_diff,
    "test_accuracy": acc_log,
})
logistic_df


In [ ]:
# --------- Modelo no convexo: MLP -------------
mlp_acc = []
for seed in range(5):
    clf = MLPClassifier(hidden_layer_sizes=(5,), max_iter=600, random_state=seed)
    clf.fit(X_train[:, 1:], y_train)  # sin columna de intercepto
    mlp_acc.append(accuracy_score(y_test, clf.predict(X_test[:, 1:])))

mlp_df = pd.DataFrame({
    "seed": range(5),
    "MLP_test_accuracy": mlp_acc,
})
mlp_df


## Interpretación

* **Regresión logística (convexa)**  
  - Las 5 corridas convergen al **mismo mínimo global** (‖β − β₀‖≈0).  
  - Accuracy de test idéntica → entrenamiento estable e independiente de la semilla.

* **MLP (no convexa)**  
  - Accuracy varía entre semillas → múltiples mínimos locales, sensibilidad a la inicialización.

Esta evidencia práctica conecta la *convexidad* de la familia exponencial (Proposición 2.1) con beneficios reales en ciencia de datos: **unicidad del MLE, reproducibilidad y optimización fiable**.